In [ ]:
import numpy

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

import pygsw
import pygetm._pygotm

In [ ]:
# Configure a water column with equal layer thicknesses
D = 5000
nlev = 100

h_ = numpy.full((nlev + 1,), D / nlev)   # layer heights plus a dummy value as very first element (needed for GOTM)
h = h_[1:]                               # actual layer heights (without the dummy value)
z_if = numpy.linspace(-D, 0., nlev + 1)  # depth of layer interfaces
z = 0.5 * (z_if[:-1] + z_if[1:])         # depth of layer centers

In [ ]:
# Try GOTM mixing logic on a random tracer field with constant diffusivity
# This sets up a figure with the tracer state, which can be advanced [time-integrated] by the following cell.

mix = pygetm._pygotm.Mixing(nlev)
mix.nuh[:] = 1e-1
y = numpy.random.random_sample(mix.nuh.size)

iday = 0
fig, ax = matplotlib.pyplot.subplots()
line, = ax.plot(y[1:], z, 'o')
title = ax.set_title('day 0')
ax.set_xlabel('tracer')
ax.set_ylabel('depth (m)')
ax.grid()

In [ ]:
# Time-integrate the vertical diffusion equation for 100 days and update the above figure while doing so.
for _ in range(100):
    iday += 1
    for _ in range(24):
        mix.diffuse(z.size, 3600., h_, y)
    line.set_xdata(y[1:])
    title.set_text('day %i' % iday)
    fig.canvas.draw()

In [ ]:
# Now set up simple temperature and salinity profile (stable temperature but randomly perturbed salinity, so there is something to diffuse):
# Start from in-situ temperature and practical salinity, then use GWS to compute other temperature and salinity metrics and plot the result.

# In-situ temperature and practical salinity
t = numpy.full_like(z, 5.)
t[80:] = 10.
sp = numpy.full_like(z, 35.) + numpy.random.random_sample(t.shape)

# Location needed by GWS
lon = numpy.full_like(z, 0.)
lat = numpy.full_like(z, 50.)
p = -z

# Absolute salinity - note we add a dummy first value so we can later use this array for pygotm mixing
SA_ = numpy.empty_like(h_)
SA = SA_[1:]
pygsw.sa_from_sp(lon, lat, p, sp, SA)

# Potential temperature
pt = numpy.empty_like(t)
pygsw.pt0_from_t(SA, t, p, pt)

# Conservative temperature - note we add a dummy first value so we can later use this array for pygotm mixing
ct_ = numpy.empty_like(SA_)
ct = ct_[1:]
pygsw.ct_from_pt(SA, pt, ct)

# Buoyancy frequency
NN = numpy.zeros((h.shape[0] + 1,), dtype=h.dtype)
pygsw.nsquared(h[:, None, None], SA[:, None, None], ct[:, None, None], p[:, None, None], lat[0, None, None], NN[1:-1, None, None])

# Plot
fig, (ax1, ax2, ax3) = matplotlib.pyplot.subplots(ncols=3, figsize=(12, 5), sharey=True)
ax1.plot(t, z, '-', label='in-situ temperature')
ax1.plot(pt, z, '-', label='potential temperature')
line_ct, = ax1.plot(ct, z, '-', label='conservative temperature')
ax1.set_xlabel('temperature (degrees C)')
ax1.set_ylabel('depth (m)')
ax1.legend()
ax1.grid()

ax2.plot(sp, z, '-', label='practical salinity')
line_SA, = ax2.plot(SA, z, '-', label='absolute salinity')
ax2.set_xlabel('salinity')
ax2.legend()
ax2.grid()

line_NN, = ax3.plot(NN, z_if, '-')
ax3.set_xlabel('buoyancy frequency squared (s-2)')
ax3.grid()

In [ ]:
# Set up a figure that shows the model state (conservative temperature, absolute salinity, buoyancy frequency, turbulent diffusivity)
# This figure can then be updated while the simulation progresses [next cell]

iday = 0
mix = pygetm._pygotm.Mixing(z.size)

fig, (ax1, ax2, ax3, ax4) = matplotlib.pyplot.subplots(ncols=4, figsize=(12, 5), sharey=True)
line_ct, = ax1.plot(ct, z, '-', label='conservative temperature')
ax1.set_xlabel('conservative temperature (degrees C)')
ax1.set_ylabel('depth (m)')
ax1.grid()

line_SA, = ax2.plot(SA, z, '-', label='absolute salinity')
ax2.set_xlabel('absolute salinity')
ax2.grid()

line_NN, = ax3.plot(NN, z_if, '-')
ax3.set_xlabel('buoyancy frequency squared (s-2)')
ax3.grid()

line_nuh, = ax4.semilogx(mix.nuh, z_if, '-')
ax4.set_xlabel('turbulent diffusivity (m2 s-1)')
ax4.set_xlim(1e-10, 10)
ax4.grid()

title = fig.suptitle('day 0')

dt = 3600.
SS = numpy.zeros_like(NN)

iday = 0
def step():
    global iday
    pygsw.nsquared(h[:, None, None], SA[:, None, None], ct[:, None, None], p[:, None, None], lat[0, None, None], NN[1:-1, None, None])
    mix.turbulence(z.size, dt, h_, D, 0., 0., 0., 0., NN, SS)
    mix.diffuse(z.size, dt, h_, SA_)
    mix.diffuse(z.size, dt, h_, ct_)
    iday += dt / (24 * 3600.)

def plot():
    line_ct.set_xdata(ct)
    line_SA.set_xdata(SA)
    line_NN.set_xdata(NN)
    line_nuh.set_xdata(mix.nuh)
    title.set_text('day %.2f' % iday)
    fig.canvas.draw()

In [ ]:
# Try a single model iteration (1 hour) and update the figure
step()
plot()

In [ ]:

# Simulate for 5 days
for _ in range(5):
    for _ in range(24):
        step()
        plot()
